# 🌍 First Day Seismology: Introduction to Earthquake Data Analysis

Welcome! This notebook will guide you through your first seismology analysis.

## 🎯 Learning Goals
- Download real earthquake data
- Visualize seismic waveforms
- Process and filter data
- Use AI assistance for learning

## 💡 How to Use This Notebook
1. Run each cell by pressing `Shift + Enter`
2. Read the explanations between code cells
3. Try modifying the code to explore
4. Ask Gemini AI if you get stuck!

---

## Step 1: Install Required Libraries

Run this cell first if you're using Google Colab:

In [ ]:
# Only run this in Google Colab
!pip install obspy -q
print("✅ Installation complete!")

## Step 2: Import Libraries

Let's import the tools we need:

In [ ]:
# Core seismology library
from obspy import UTCDateTime
from obspy.clients.fdsn import Client

# Plotting and numerical tools
import matplotlib.pyplot as plt
import numpy as np

# Configure plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ All libraries imported successfully!")
print("🎉 You're ready to analyze earthquakes!")

## Step 3: Connect to IRIS Data Center

IRIS provides access to seismic data from around the world:

In [ ]:
# Create client connection
client = Client("IRIS")
print("📡 Connected to IRIS data center!")
print("🌐 You can now download data from thousands of stations worldwide")

## Step 4: Download Real Earthquake Data

Let's get data from a significant earthquake. Try this example or ask AI to find a recent earthquake!

In [ ]:
# Define earthquake time and station
# Example: 2024 Turkey Earthquake
earthquake_time = UTCDateTime("2024-02-06T01:17:00")
duration = 600  # seconds (10 minutes)

print(f"⏳ Downloading data for earthquake at {earthquake_time}...")

# Download waveform data
stream = client.get_waveforms(
    network="IU",           # Global network
    station="ANTO",         # Ankara, Turkey
    location="00",          # Location code
    channel="BHZ",          # Broadband vertical
    starttime=earthquake_time,
    endtime=earthquake_time + duration
)

print(f"✅ Success! Downloaded {len(stream)} trace(s)")
print("\n📊 Data Information:")
print(stream)

## Step 5: Plot the Raw Data

Let's see what the earthquake looks like:

In [ ]:
# Quick plot using ObsPy's built-in plotting
stream.plot(figsize=(12, 4))

### 🤔 What Do You See?

- **Before the earthquake:** Small background noise (microseisms)
- **P-wave arrival:** First sharp spike (compressional wave)
- **S-wave arrival:** Larger amplitude waves (shear wave)
- **Surface waves:** Long-period oscillations

💡 **Try asking Gemini:** "What are the differences between P, S, and surface waves?"

## Step 6: Get Detailed Statistics

In [ ]:
# Extract the trace (single recording)
trace = stream[0]

print("📊 Recording Details:")
print(f"  Station: {trace.stats.station}")
print(f"  Channel: {trace.stats.channel}")
print(f"  Sampling rate: {trace.stats.sampling_rate} Hz")
print(f"  Number of samples: {trace.stats.npts}")
print(f"  Start time: {trace.stats.starttime}")
print(f"  End time: {trace.stats.endtime}")
print(f"  Duration: {trace.stats.endtime - trace.stats.starttime:.1f} seconds")

## Step 7: Process the Data

Let's clean up the data by filtering:

In [ ]:
# Make a copy to preserve original data
stream_filtered = stream.copy()

# Apply bandpass filter (0.1 to 10 Hz)
print("🔧 Applying bandpass filter (0.1 - 10 Hz)...")
stream_filtered.filter("bandpass", freqmin=0.1, freqmax=10.0)

print("✅ Filtering complete!")
print("This removes high-frequency noise and long-period drift.")

## Step 8: Compare Raw vs Filtered

In [ ]:
# Create comparison plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8))

# Plot raw data
times = stream[0].times()
ax1.plot(times, stream[0].data, 'b-', linewidth=0.5, alpha=0.8)
ax1.set_title('Raw Seismic Data', fontsize=14, fontweight='bold')
ax1.set_ylabel('Amplitude (counts)', fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot filtered data
ax2.plot(times, stream_filtered[0].data, 'r-', linewidth=0.5, alpha=0.8)
ax2.set_title('Filtered Seismic Data (0.1-10 Hz)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Time (seconds)', fontsize=11)
ax2.set_ylabel('Amplitude (counts)', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📈 Notice how filtering makes the earthquake signals clearer!")

## Step 9: Calculate Basic Parameters

In [ ]:
# Get the data array
data = stream_filtered[0].data

# Calculate statistics
max_amp = np.max(np.abs(data))
mean_val = np.mean(data)
std_val = np.std(data)

# Find time of maximum amplitude
max_index = np.argmax(np.abs(data))
max_time = times[max_index]

print("📊 Waveform Statistics:")
print(f"  Maximum amplitude: {max_amp:.2e}")
print(f"  Mean value: {mean_val:.2e}")
print(f"  Standard deviation: {std_val:.2e}")
print(f"  Peak arrival time: {max_time:.2f} seconds")

## Step 10: Zoom In on Wave Arrivals

In [ ]:
# Plot first 200 seconds to see wave arrivals clearly
fig, ax = plt.subplots(figsize=(14, 6))

# Filter for first 200 seconds
mask = times <= 200
ax.plot(times[mask], data[mask], 'b-', linewidth=1)

# Mark potential wave arrivals (these are estimates!)
ax.axvline(x=50, color='r', linestyle='--', linewidth=2, 
           label='~P-wave arrival', alpha=0.7)
ax.axvline(x=90, color='g', linestyle='--', linewidth=2, 
           label='~S-wave arrival', alpha=0.7)

ax.set_title('Wave Arrivals - First 200 Seconds', fontsize=14, fontweight='bold')
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("🌊 P-waves (Primary) arrive first - they're faster!")
print("🌊 S-waves (Secondary) arrive later - they're slower but stronger!")

## 🎉 Congratulations!

You've completed your first seismology analysis! You've learned to:
- ✅ Download real earthquake data
- ✅ Visualize seismograms
- ✅ Filter seismic signals
- ✅ Calculate basic parameters
- ✅ Identify wave arrivals

---

## 🚀 Challenge Exercises

Try these modifications to deepen your understanding:

### Easy:
1. Change the station to `"ANMO"` (New Mexico, USA)
2. Download 20 minutes of data instead of 10
3. Try different filter frequencies (e.g., 1-5 Hz)

### Medium:
4. Download all three components (BHE, BHN, BHZ)
5. Plot them on the same figure
6. Calculate the time difference between P and S waves

### Advanced:
7. Create a spectrogram to see frequency content
8. Download data from multiple stations
9. Compare waveforms at different distances

---

## 🤖 Using AI to Learn More

Ask Gemini these questions:
- "How do I create a spectrogram in ObsPy?"
- "What's the relationship between S-P time and distance?"
- "How can I download data from multiple stations at once?"

Use NotebookLM:
- Upload a seismology textbook chapter
- Ask: "Explain earthquake wave propagation"
- Generate a study guide for this notebook

---

## 📚 Additional Resources

- **ObsPy Documentation:** https://docs.obspy.org/
- **IRIS Data Services:** https://www.iris.edu/
- **Recent Earthquakes:** https://earthquake.usgs.gov/
- **Setup Guide:** See SETUP_GUIDE.md in the repository
- **Full Tutorial:** See EXAMPLE_TUTORIAL.md for more details

---

**Happy coding! 🌍📊🐍**